# 分子の比熱を計算するスクリプト
* gen_lammps_inputs.ipynb
1. SMILESから分子生成
2. LigParGenからOPLS取得
3. OPLS formatのチェック・修正
4. PFP-tuned OPLSの取得
5. LAMMPS入力ファイルの生成

* setup_lammps.ipynb
6. LAMMPS実行フォルダの作成
7. SMILES、JOB番号、生成日時、LAMMPS設定をメタデータとして保存
8. LAMMPS入力ファイルを実行フォルダにコピー
9. LAMMPS実行スクリプトの生成

* run.ipynb
10. LAMMPS実行
11. LAMMPS終了の確認、異常終了-> 12､正常終了->13
12. エラーの確認、原因を出力して終了（LAMMPS入力を修正して再実行）
13. LAMMPS出力ファイルの読み込み、解析用データの保存

* analyze.ipynb
14. LAMMPS計算から比熱の計算
15. 計算結果の解析



In [5]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


cwd = os.getcwd()
rootdir = cwd.rsplit("jobs", 1)[0]
modulepaths = [os.path.join(rootdir, "nbpackage")]
print(modulepaths)
sys.path.extend(modulepaths)
import nbpackage
from nbpackage.AN import pfp
from nbpackage.AN import opt
from nbpackage.AN import ioutils
from nbpackage.AN import rwmol
from nbpackage.AN import view
from nbpackage.AN import md

['/home/jovyan/Jobs/01_EV_Coulant/20220524_specific_heats/nbpackage']
importing Jupyter notebook from /home/jovyan/Jobs/01_EV_Coulant/20220524_specific_heats/nbpackage/nbpackage/AN/rwmol.ipynb
importing Jupyter notebook from /home/jovyan/Jobs/01_EV_Coulant/20220524_specific_heats/nbpackage/nbpackage/AN/view.ipynb


In [3]:

import ase
from ase import Atoms, units
from ase.constraints import ExpCellFilter
from ase.visualize import view
from ase.optimize import BFGS

from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator

estimator = Estimator()
calculator = ASECalculator(estimator)

1. SMILESから分子生成

In [ ]:
smileses = ["CC", "CCc1ccc(cc1)C"]
rwmols = rwmol.get_rwmols_from_smiles(smiles=smileses)

2. LigParGenからOPLS取得

In [1]:

lammpsdatafiles = []
for i, s in enumerate(smileses):
    _filename = f"{i}.data"
    md.ligpargen.get_opls_from_smiles(smiles=s, format="lammps", to=_filename)
    lammpsdatafiles.append(_filename)


NameError: name 'smileses' is not defined

3. OPLSフォーマットの確認・修正 

In [ ]:

for _filename in lammpsdatafiles:
    _lines = md.lammps.check_ligpargen_format(_filename)
    with open(_filename, "w") as g:
        g.write("\n".join(_lines) + "\n")


4. PFP-tuned OPLS取得

In [2]:
filenames = []
for _filename in lammpsdatafiles:
    _filename = os.path.splitext(_filename)
    _filename = f"{_filename[0]}_pfpfit{_filename[1]}"
    md.refine_opls_via_pfp(_filename, to=_filename)
    filenames.append(_filename)

SyntaxError: EOL while scanning string literal (<ipython-input-2-a4bae17df178>, line 4)

5. LAMMPS入力ファイルの作成